In [4]:
import os
os.getcwd()

'/home/xinleilin/Projects/Video_Pose/3_VideoMambaPose/src/models/experiments/mamba_encoder'

In [5]:
os.chdir("/home/linxin67/projects/def-btaati/linxin67/Projects/MambaPose/Video_Pose/3_VideoMambaPose/src/models/experiments/mamba_encoder")
os.getcwd()

FileNotFoundError: [Errno 2] No such file or directory: '/home/linxin67/projects/def-btaati/linxin67/Projects/MambaPose/Video_Pose/3_VideoMambaPose/src/models/experiments/mamba_encoder'

In [ ]:
!pip list

Package                  Version
------------------------ ----------
asttokens                2.4.1
certifi                  2024.2.2
charset-normalizer       3.3.2
comm                     0.2.2
debugpy                  1.6.7
decorator                5.1.1
exceptiongroup           1.2.0
executing                2.0.1
filelock                 3.14.0
fsspec                   2024.3.1
idna                     3.7
importlib_metadata       7.1.0
ipykernel                6.29.3
ipython                  8.22.2
jedi                     0.19.1
Jinja2                   3.1.4
jupyter_client           8.6.1
jupyter_core             5.7.2
MarkupSafe               2.1.5
matplotlib-inline        0.1.7
mpmath                   1.3.0
nest_asyncio             1.6.0
networkx                 3.3
numpy                    1.26.4
nvidia-cublas-cu12       12.1.3.1
nvidia-cuda-cupti-cu12   12.1.105
nvidia-cuda-nvrtc-cu12   12.1.105
nvidia-cuda-runtime-cu12 12.1.105
nvidia-cudnn-cu12        8.9.2.26
nvidia-cuf

In [1]:
import os
import torch
import torch.nn as nn
from functools import partial
from torch import Tensor
from typing import Optional
import torch.utils.checkpoint as checkpoint
 
# remember that this is einstein operation, which is the special fancy way of reshaping.
from einops import rearrange
from timm.models.vision_transformer import _cfg
from timm.models.registry import register_model
from timm.models.layers import trunc_normal_

from timm.models.layers import DropPath, to_2tuple
from timm.models.vision_transformer import _load_weights

import math

from mamba_ssm.modules.mamba_simple import Mamba



/home/xinleilin/Programs/miniconda3/envs/mamba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# making sure the data and models are on the GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model.to(device)
# data = data.to(device)

In [3]:
# generating a random input
# (Batch, Channel number, NumFrames, W, H) = (16, 3, 8, 224, 224)

# Define the dimensions
batch_size = 16
num_frames = 8
height = 224
width = 224
channels = 3

# Generate a random tensor
# RuntimeError: Expected 4D (unbatched) or 5D (batched) input to conv3d
# https://discuss.pytorch.org/t/what-should-be-the-input-shape-for-3d-cnn-on-a-sequence-of-images/111884
test_video = torch.rand(batch_size, channels, num_frames, height, width) # I get an error .... 384, 3, 1, 16, 16

# Check the shape of the random tensor
print("Shape of the random tensor:", test_video.shape)


Shape of the random tensor: torch.Size([16, 3, 8, 224, 224])


#### Output layers inspired by DeciWatch

#### running the mamba model

In [4]:
import VideoMamba as vm

In [5]:
class VideoMambaPose(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = vm.videomamba_tiny() # TODO this is temporary, because I don't wnat to overrun the GPU memory

    def forward(self, x):
        x = self.encoder(x)

        # adding my own layer, see how deciwatch did it.
        print(x.shape)
        return x


In [10]:
test_model = VideoMambaPose()

# move the data to the GPU
test_model = test_model.to(device)
test_video = test_video.to(device)

y = test_model(test_video)

Use checkpoint: False
Checkpoint number: 0
torch.Size([16, 1000])


In [7]:
print(test_video.shape)

# the output is a 1 dimensional tensor (i guess it was flattened)
print(y.shape)

torch.Size([16, 3, 8, 224, 224])
torch.Size([16, 1000])


In [11]:


print(test_video.shape)

print(y)



torch.Size([16, 3, 8, 224, 224])
tensor([[-0.0915, -0.0965,  0.3975,  ..., -0.2715,  0.0296,  0.8478],
        [-0.0867, -0.0659,  0.3894,  ..., -0.3031,  0.0429,  0.6752],
        [-0.1017,  0.0498,  0.4555,  ..., -0.3780,  0.1321,  0.7621],
        ...,
        [-0.2443, -0.0894,  0.2556,  ..., -0.2460,  0.1110,  0.6184],
        [-0.1089,  0.0173,  0.4038,  ..., -0.2949,  0.0652,  0.6235],
        [-0.0705, -0.0685,  0.4025,  ..., -0.3186,  0.1360,  0.6528]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
